接下來 就是要根據吳XX 說 標記8個點 先算直線方程

1.讀取原始 畫直線的同時 ""求方程係數""

2.讀取上次Binary後得影像 做 中間雕像遮罩 並"""把剩餘的所有點座標都存起來""" 

3.開始"""算每個點與這四條線的距離""" 設定四條線的鄰居"""threadhold距離"""

(3和4其實可以同時做)

4.用四個矩陣 求得四group的點之後 取平均ㄅ 這樣就會有homography的四個座標頂點了

Part 1 求直線方程式的"vx, vy, x0, y0" 

畫方框圖只是示意 但是還是做一下 比較心安 

In [31]:
# '''畫方框圖只是示意 但是還是做一下 比較心安 現在不需要了 其實下面已經有寫純用8點計算直線參數了'''
# import cv2
# import numpy as np
# import os

# # 檔案路徑設定
# #=========================================================

# # 輸入
# input_directory = "ShadowStrip"
# input_file_name = "0000.jpg"

# # 輸出
# output_directory = "box_frame_line"
# output_file_name = "box_frame_line.jpg"
# if not os.path.exists(output_directory):
#     os.makedirs(output_directory)

# #==========================================================

# # 讀取圖像
# image_path = os.path.join(input_directory, input_file_name)
# image = cv2.imread(image_path)

# # 定義八個座標點
# points = [(238, 67), (988, 123), (270, 998), (959, 849),
#           (72,157), (619,181), (103,760), (617, 690)]

# # 在圖像上畫四條線段
# for i in range(4):
#     cv2.line(image, points[i*2], points[(i*2)+1], (0, 0, 255), 2)
# # 在點上標上數字
# for i in range(8):
#     cv2.putText(image, str((i+1)), (points[i][0] - 30, points[i][1] - 20),
#                 cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
#     cv2.circle(image, points[i], 10, (0, 255, 0), -1)  # 在座標點位置繪製綠色實心圓

# # 計算直線方程式的係數
# for i in range(4):
#     x1, y1 = points[i*2]
#     x2, y2 = points[(i*2)+1]
#     vx, vy, x0, y0 = cv2.fitLine(np.array([[x1, y1], [x2, y2]], dtype=np.int32), cv2.DIST_L2, 0, 0.01, 0.01)
#     slope = vy / vx
#     intercept = y0 - (slope * x0)
#     print(f"Line {i+1} equation: y = {slope}x + {intercept}")
#     print(" vx = ",vx,"vy = ",vy,"x0 = ",x0,"y0 = ",y0)

# # 儲存結果
# output_filename = os.path.join(output_directory, input_file_name.replace('.jpg', '_' + output_file_name))
# cv2.imwrite(output_filename, image)

# print("All images processed Mask result successfully.")
# print(f"Binary image of red regions saved at {output_directory}")


Line 1 equation: y = [0.07466667]x + [49.229332]
 vx =  [0.99722403] vy =  [0.0744594] x0 =  [613.] y0 =  [95.]
Line 2 equation: y = [-0.21625544]x + [1056.3889]
 vx =  [0.97740626] vy =  [-0.21136943] x0 =  [614.5] y0 =  [923.5]
Line 3 equation: y = [0.04387569]x + [153.84094]
 vx =  [0.9990389] vy =  [0.04383352] x0 =  [345.5] y0 =  [169.]
Line 4 equation: y = [-0.13618676]x + [774.0272]
 vx =  [0.9908536] vy =  [-0.13494115] x0 =  [360.] y0 =  [725.]
All images processed Mask result successfully.
Binary image of red regions saved at box_frame_line


Part2:現在要處裡 原本二值化 儲存好的照片 目標是獲得四個homography的頂點

單張圖片 ""blocked雕像"" 版本 目前不需要他 已有for版本 下面

In [26]:
# '''單張圖片blocked 中間雕像版本 目前不需要他 已有for版本 下面'''
# # 檔案路徑設定
# #=========================================================

# # 輸入
# input_directory = "RGB_threshold_RedBinary"
# input_file_name = "0000_RGB_threshold_RedBinary.jpg"

# # 輸出
# output_directory = "statue_blocked"
# output_file_name = "statue_blocked.jpg"
# if not os.path.exists(output_directory):
#     os.makedirs(output_directory)

# #==========================================================

# # 讀取圖像
# image_path = os.path.join(input_directory, input_file_name)
# image = cv2.imread(image_path)

# # 定義方框範圍的最左上與最右下座標
# top_left = (365, 235)
# bottom_right = (600, 680)

# # 將指定範圍內的像素值設為全黑
# image[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]] = 0

# # 儲存結果
# output_filename = os.path.join(output_directory, input_file_name.replace('.jpg', '_' + output_file_name))
# cv2.imwrite(output_filename, image)

# print(f"Processed image saved at {output_filename}")


遍歷版本 把所有雕像都block掉 output_directory = "statue_blocked"

In [1]:
'''遍歷版本 把所有雕像都block掉'''
import cv2
import os

# 檔案路徑設定
#=========================================================

# 輸入目錄
input_directory = "RGB_threshold_RedBinary"

# 輸出目錄
output_directory = "statue_blocked"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

#==========================================================

# 遍歷輸入目錄中的所有檔案
for filename in os.listdir(input_directory):
    if filename.endswith(".jpg"):  # 確保只處理 .jpg 檔案
        # 讀取圖像
        input_file_path = os.path.join(input_directory, filename)
        image = cv2.imread(input_file_path)

        # 定義方框範圍的最左上與最右下座標
        top_left = (365, 235)
        bottom_right = (600, 680)

        # 將指定範圍內的像素值設為全黑
        image[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]] = 0

        # 儲存結果
        output_filename = os.path.join(output_directory, filename.replace('.jpg', '_statue_blocked.jpg'))
        cv2.imwrite(output_filename, image)

        # print(f"Processed image saved at {output_filename}")


Part 3 現在要處理 每張已經blocked中間雕像的圖 剩餘的白色 分別屬於哪一個線的管轄範圍了

純粹根據8點 求四條線的直線方程的係數 用一個向量好好儲存起來 並且建立好距離公式

然後 讀取舊有已經blocked雕像的照片 開始算所有非黑的距離

求直線距離 純粹8點 純數學版本 求係數(- line_params: 包含直線參數的向量 (vx, vy, x0, y0)。)

In [32]:
# """求直線距離 純粹8點 純數學版本 求係數(- line_params: 包含直線參數的向量 (vx, vy, x0, y0)。)"""

# import cv2
# import numpy as np

# def distance_to_line(x_prime, y_prime, line_params):
#     """
#     計算其他座標點到直線的距離。

#     參數:
#     - x_prime: 其他座標點的 x 座標。
#     - y_prime: 其他座標點的 y 座標。
#     - line_params: 包含直線參數的向量 (vx, vy, x0, y0)。

#     返回:
#     - 該座標點到直線的距離。
#     """
#     vx, vy, x0, y0 = line_params
#     distance = np.abs((x_prime - x0) * vy - (y_prime - y0) * vx) / np.sqrt(vx**2 + vy**2)
#     return distance

# # 定義八個座標點
# points = [(238, 67), (988, 123), (270, 998), (959, 849),
#           (72,157), (619,181), (103,760), (617, 690)]

# # 存儲直線方程的係數向量
# line_params_list = []

# # 計算直線方程式的係數
# for i in range(4):
#     x1, y1 = points[i*2]
#     x2, y2 = points[(i*2)+1]
#     vx, vy, x0, y0 = cv2.fitLine(np.array([[x1, y1], [x2, y2]], dtype=np.int32), cv2.DIST_L2, 0, 0.01, 0.01)

#     # slope = vy / vx
#     # intercept = y0 - (slope * x0)
#     # print(f"Line {i+1} equation: y = {slope}x + {intercept}")
#     # print(" vx = ",vx,"vy = ",vy,"x0 = ",x0,"y0 = ",y0)

    
#     # 存儲到向量中
#     line_params_list.append((vx, vy, x0, y0))

# # 在 for 迴圈以外使用直線參數
# for i, line_params in enumerate(line_params_list):
#     print(f"Line {i+1} parameters:", line_params)

# # some_other_points 
# # # 在自定義函數中使用直線參數
# # for x_prime, y_prime in some_other_points:
# #     distance = distance_to_line(x_prime, y_prime, line_params)
# #     print(f"Distance from point ({x_prime}, {y_prime}) to line:", distance)



Line 1 parameters: (array([0.99722403], dtype=float32), array([0.0744594], dtype=float32), array([613.], dtype=float32), array([95.], dtype=float32))
Line 2 parameters: (array([0.97740626], dtype=float32), array([-0.21136943], dtype=float32), array([614.5], dtype=float32), array([923.5], dtype=float32))
Line 3 parameters: (array([0.9990389], dtype=float32), array([0.04383352], dtype=float32), array([345.5], dtype=float32), array([169.], dtype=float32))
Line 4 parameters: (array([0.9908536], dtype=float32), array([-0.13494115], dtype=float32), array([360.], dtype=float32), array([725.], dtype=float32))


現在已經有計算與四條直線的距離公式解function了 你能幫我再添加以下功能與步驟:
1. 遍歷讀取叫做statue_blocked的目錄 裡面所有的.jpg檔 用openCV讀入
2. 讀近來的這些照片 都是二值化的照片 我想把所有白色的點座標(x',y') 都套入距離公式 只要該白色座標點  離四條的其中一條 與哪一條最近 就歸類並紀錄成一群 (算是變成該線的管轄)
我想用計算所有白色座標點計算與四線距離 把所有座標儲存分成四類 分成距離四條線最近的一群 
所以會想用向量儲存 四個存(x',y')座標對(pair)  分別儲存四群座標
3. 計算每一群的平均值 產生一個平均的座標點 結果會有四個(avg_x, avg_y) 因為有四群

好的，那我們只處理一張照片。這裡是修改後的程式碼：

這段程式會讀取 statue_blocked.jpg 圖像，找出其中的白色點，然後根據這些白色點到四條直線的距離，將它們分組到四個群組中。最後，計算每個群組的平均座標點並印出。

In [54]:
# """ ""單張照片""平均座標點旁邊都會標上相應的群組編號。"""

# import cv2
# import numpy as np

# def distance_to_line(x_prime, y_prime, line_params):
#     """
#     計算其他座標點到直線的距離。

#     參數:
#     - x_prime: 其他座標點的 x 座標。
#     - y_prime: 其他座標點的 y 座標。
#     - line_params: 包含直線參數的向量 (vx, vy, x0, y0)。

#     返回:
#     - 該座標點到直線的距離。
#     """
#     vx, vy, x0, y0 = line_params
#     distance = np.abs((x_prime - x0) * vy - (y_prime - y0) * vx) / np.sqrt(vx**2 + vy**2)
#     return distance

# # 定義八個座標點
# points = [(238, 67), (988, 123), (270, 998), (959, 849),
#           (72,157), (619,181), (103,760), (617, 690)]

# # 存儲直線方程的係數向量
# line_params_list = []

# # 計算直線方程式的係數
# for i in range(4):
#     x1, y1 = points[i*2]
#     x2, y2 = points[(i*2)+1]
#     vx, vy, x0, y0 = cv2.fitLine(np.array([[x1, y1], [x2, y2]], dtype=np.int32), cv2.DIST_L2, 0, 0.01, 0.01)
#     line_params_list.append((vx, vy, x0, y0))
# # ======================================

# # 建立輸入目錄
# input_directory = "statue_blocked"
# input_file_name = "0031_RGB_threshold_RedBinary_statue_blocked.jpg"

# # 建立輸出目錄
# output_directory = "box_frame_4_points"
# output_file_name = "box_frame_4_points.jpg"
# if not os.path.exists(output_directory):
#     os.makedirs(output_directory)
# # ========================================

# # 讀取圖像
# image_path = os.path.join(input_directory, input_file_name)
# image = cv2.imread(image_path)

# # 找出所有白色點的座標
# white_points = np.where(image == 255)
# white_points = np.column_stack((white_points[1], white_points[0]))  # 將座標點轉換成 (x, y) 格式

# # 初始化四個群組
# grouped_coordinates = [[] for _ in range(4)]

# # 將白色點分組
# for point in white_points:
#     distances = [distance_to_line(point[0], point[1], line_params) for line_params in line_params_list]     # 行使用了列表理解，遍歷了 line_params_list 中的每一組直線參數，計算出每個白色點到這四條直線的距離，並將這些距離存儲在 distances 列表中。
#     min_distance_index = np.argmin(distances)       # 這一行程式碼是找到 distances 列表中距離最小的那個值的""索引"""，也就是找到了這個白色點距離四條直線中哪一條直線最近。
#     grouped_coordinates[min_distance_index].append(point)   # 這一行程式碼是將這個白色點添加到距離最近的那條直線所對應的群組中。grouped_coordinates 是一個列表，其中的每個元素代表一個群組，索引 min_distance_index 對應到距離最小的那條直線。然後，append() 方法將這個白色點 point 添加到這個群組中。

# # 計算每個群組的平均值
# avg_coordinates = [(np.mean(group, axis=0)[0], np.mean(group, axis=0)[1]) for group in grouped_coordinates]

# # 印出平均座標點
# for i, avg_coord in enumerate(avg_coordinates):
#     print(f"Average coordinates for group {i+1}: {avg_coord}")


# # 在圖像上畫出平均座標點並標上群組編號
# cross_size = (50, 50)  # 十字的大小，(橫向大小, 縱向大小)
# font = cv2.FONT_HERSHEY_SIMPLEX
# font_scale = 1
# font_thickness = 1
# for i, coord in enumerate(avg_coordinates, 1):
#     x, y = int(coord[0]), int(coord[1])
#     cross_half_size = (cross_size[0] // 2, cross_size[1] // 2)
#     cv2.line(image, (x - cross_half_size[0], y), (x + cross_half_size[0], y), (0, 255, 0), 1)  # 橫向線
#     cv2.line(image, (x, y - cross_half_size[1]), (x, y + cross_half_size[1]), (0, 255, 0), 1)  # 縱向線
#     cv2.putText(image, f"Group {i}", (x + 15, y - 15), font, font_scale, (0, 255, 0), font_thickness, cv2.LINE_AA)


# # 儲存結果
# output_filename = os.path.join(output_directory, input_file_name.replace('.jpg', '_' + output_file_name))
# cv2.imwrite(output_filename, image)


Average coordinates for group 1: (684.3953488372093, 100.02325581395348)
Average coordinates for group 2: (675.6585365853658, 909.5121951219512)
Average coordinates for group 3: (386.8, 170.3)
Average coordinates for group 4: (397.0, 719.2307692307693)


True

平均座標點旁邊都會標上相應的群組編號。 遍歷版本 

In [2]:
"""平均座標點旁邊都會標上相應的群組編號。 遍歷版本 """

import cv2
import numpy as np
import os

def distance_to_line(x_prime, y_prime, line_params):
    """
    計算其他座標點到直線的距離。

    參數:
    - x_prime: 其他座標點的 x 座標。
    - y_prime: 其他座標點的 y 座標。
    - line_params: 包含直線參數的向量 (vx, vy, x0, y0)。

    返回:
    - 該座標點到直線的距離。
    """
    vx, vy, x0, y0 = line_params
    distance = np.abs((x_prime - x0) * vy - (y_prime - y0) * vx) / np.sqrt(vx**2 + vy**2)
    return distance

# 定義八個座標點
points = [(238, 67), (988, 123), (270, 998), (959, 849),
          (72,157), (619,181), (103,760), (617, 690)]

# 存儲直線方程的係數向量
line_params_list = []

# !!!!!!!!!儲存  遍歷所有照片結果 平均四點座標 的大矩陣!!!!!!!!!!!!!!!
avg_coordinates_matrix = []

# 計算直線方程式的係數
for i in range(4):
    x1, y1 = points[i*2]
    x2, y2 = points[(i*2)+1]
    vx, vy, x0, y0 = cv2.fitLine(np.array([[x1, y1], [x2, y2]], dtype=np.int32), cv2.DIST_L2, 0, 0.01, 0.01)
    line_params_list.append((vx, vy, x0, y0))

# 建立輸入目錄
input_directory = "statue_blocked"

# 建立輸出目錄
output_directory = "box_frame_4_points"
output_file_name = "box_frame_4_points.jpg"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# 遍歷目錄中的所有.jpg檔案
for filename in os.listdir(input_directory):
    if filename.endswith(".jpg"):
        # 讀取圖像
        image_path = os.path.join(input_directory, filename)
        image = cv2.imread(image_path)

        # 找出所有白色點的座標
        white_points = np.where(image == 255)
        white_points = np.column_stack((white_points[1], white_points[0]))  # 將座標點轉換成 (x, y) 格式

        # 初始化四個群組
        grouped_coordinates = [[] for _ in range(4)]

        # 將白色點分組
        for point in white_points:
            distances = [distance_to_line(point[0], point[1], line_params) for line_params in line_params_list]
            min_distance_index = np.argmin(distances)
            grouped_coordinates[min_distance_index].append(point)

        # 計算每個群組的平均值
        avg_coordinates = [(np.mean(group, axis=0)[0], np.mean(group, axis=0)[1]) for group in grouped_coordinates]

        # 儲存平均四點座標到矩陣中
        # !!!!!!!!!儲存  遍歷所有照片結果 平均四點座標 的大矩陣!!!!!!!!!!!!!!!
        avg_coordinates_matrix.append(avg_coordinates)

# ================以下只是畫圖======儲存標記四個點=================
        # 在圖像上畫出平均座標點並標上群組編號
        cross_size = (50, 50)  # 十字的大小，(橫向大小, 縱向大小)
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1
        font_thickness = 1
        for i, coord in enumerate(avg_coordinates, 1):
            x, y = int(coord[0]), int(coord[1])
            cross_half_size = (cross_size[0] // 2, cross_size[1] // 2)
            cv2.line(image, (x - cross_half_size[0], y), (x + cross_half_size[0], y), (0, 255, 0), 1)  # 橫向線
            cv2.line(image, (x, y - cross_half_size[1]), (x, y + cross_half_size[1]), (0, 255, 0), 1)  # 縱向線
            cv2.putText(image, f"Group {i}", (x + 15, y - 15), font, font_scale, (0, 255, 0), font_thickness, cv2.LINE_AA)

        # 儲存結果
        output_filename = os.path.join(output_directory, filename.replace('.jpg', '_' + output_file_name))
        cv2.imwrite(output_filename, image)

print(avg_coordinates_matrix)

[[(558.6603773584906, 90.50943396226415), (560.2608695652174, 935.0652173913044), (295.2352941176471, 166.64705882352942), (310.9230769230769, 730.5384615384615)], [(558.7777777777778, 91.17777777777778), (560.725, 934.625), (295.5, 166.5), (311.7692307692308, 730.3846153846154)], [(559.6451612903226, 91.33870967741936), (560.936170212766, 934.0212765957447), (296.42857142857144, 166.64285714285714), (311.75, 730.6875)], [(560.875, 91.203125), (562.0416666666666, 934.1041666666666), (296.75, 167.16666666666666), (312.6923076923077, 730.0769230769231)], [(561.8039215686274, 90.7843137254902), (563.6382978723404, 933.4468085106383), (297.3125, 166.9375), (313.6470588235294, 730.2941176470588)], [(563.8809523809524, 91.88095238095238), (565.3913043478261, 934.1086956521739), (299.1111111111111, 166.77777777777777), (315.3333333333333, 729.9166666666666)], [(565.9803921568628, 91.2156862745098), (567.0, 933.1304347826087), (300.93333333333334, 166.86666666666667), (316.84615384615387, 729.

Part 3 現在有座標點了 把僅有雕像的結果 用homography映射到一張1080*1080的圖案上

輸入: 只有中間雕像的圖 (尚未產生 之前只有blocked雕像而已)

輸出: homo過的結果 正面 側面

In [3]:
# 定義八個座標點
points = [(238, 67), (988, 123), (270, 998), (959, 849),
          (72,157), (619,181), (103,760), (617, 690)]

In [4]:
"""!!!!!!!!!儲存  遍歷所有照片結果 平均四點座標 的大矩陣!!(55, 4, 2)"""
# 列出欄位資料型別等資訊 
np.shape(avg_coordinates_matrix) # (55, 4, 2)

(55, 4, 2)

(only1) 左視角切片 的結果

In [88]:
# "(only1) 左視角切片 的結果"

# import cv2
# import numpy as np
# import os

# # 建立輸出目錄
# output_directory = "left_view_slice"
# if not os.path.exists(output_directory):
#     os.makedirs(output_directory)

# # 定義四個點的坐標
# src_points = np.array([[684.3953488372093, 100.02325581395348],
#                        [675.6585365853658, 909.5121951219512],
#                        [386.8, 170.3],
#                        [397.0, 719.2307692307693]], dtype=np.float32)

# # 定義目標四個角的坐標（從右上、右下、左上、左下的順序）
# dst_points = np.array([[1080, 0],
#                        [1080, 1080],
#                        [0, 0],
#                        [0, 1080]], dtype=np.float32)

# # 載入原始圖像
# input_directory = "RGB_threshold_RedBinary"
# input_file_name = "0031_RGB_threshold_RedBinary.jpg"
# image_path = os.path.join(input_directory, input_file_name)
# image = cv2.imread(image_path)

# # 執行透視變換（homography）
# H, _ = cv2.findHomography(src_points, dst_points)

# # 對原始圖像應用透視變換
# warped_image = cv2.warpPerspective(image, H, (1080, 1080))


# # 儲存結果
# output_filename = os.path.join(output_directory, input_file_name.replace('.jpg', '_left_view_slice.jpg'))
# cv2.imwrite(output_filename, warped_image)

# print("Homography transformation completed and saved successfully.")


Homography transformation completed and saved successfully.


""" 所有照片 左視角切片 的結果""" 先確認一下 np.shape(avg_coordinates_matrix) # (55, 4, 2)

In [5]:
np.shape(avg_coordinates_matrix) # (55, 4, 2)

(55, 4, 2)

左視版本

In [8]:
import cv2
import numpy as np
import os

# 讀取所有.jpg文件
input_directory = "RGB_threshold_RedBinary"
output_directory = "left_view_slice"


# 檢查輸出目錄是否存在，若不存在則建立
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# 初始化 avg_coordinates_matrix
# 假設 avg_coordinates_matrix 是一個大小為 (55, 4, 2) 的 NumPy 矩陣
# 55 張照片，每張照片有 4 個點，每個點有 x 和 y 座標
# 在這裡，我們假設 avg_coordinates_matrix 已經被定義和填充
print("avg_coordinates_matrix: ", np.shape(avg_coordinates_matrix))

# 遍歷所有.jpg文件
for filename in os.listdir(input_directory):
    if filename.endswith(".jpg"):
        # 載入原始圖像
        image_path = os.path.join(input_directory, filename)
        image = cv2.imread(image_path)

        # 取出對應的平均座標點
        index = int(filename.split("_")[0])  # 假設文件名是"index_...jpg"
        avg_coordinates = avg_coordinates_matrix[index]
        avg_coordinates = np.array(avg_coordinates, dtype=np.float32)

        # 定義目標四個角的坐標（從右上、右下、左上、左下的順序）
        dst_points = np.array([[1080, 0],
                               [1080, 1080],
                               [0, 0],
                               [0, 1080]], dtype=np.float32)

        # 執行透視變換（homography）
        H, _ = cv2.findHomography(avg_coordinates, dst_points)

        # 對原始圖像應用透視變換
        warped_image = cv2.warpPerspective(image, H, (1080, 1080))

        # 儲存結果
        output_filename = os.path.join(output_directory, filename.replace('.jpg', '_left_view_slice.jpg'))
        cv2.imwrite(output_filename, warped_image)

print("Left view slice Homography transformation completed and saved successfully.")


avg_coordinates_matrix:  (55, 4, 2)
Left view slice Homography transformation completed and saved successfully.


正視版本

In [9]:
import cv2
import numpy as np
import os

# 定義四個點的坐標（左上、右上、左下、右下的順序）
src_points = np.array([[238, 67],
                       [988, 123],
                       [270, 998],
                       [959, 849]], dtype=np.float32)

# 讀取所有.jpg文件
input_directory = "RGB_threshold_RedBinary"
output_directory = "front_view_slice"


# 檢查輸出目錄是否存在，若不存在則建立
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# 遍歷所有.jpg文件
for filename in os.listdir(input_directory):
    if filename.endswith(".jpg"):
        # 載入原始圖像
        image_path = os.path.join(input_directory, filename)
        image = cv2.imread(image_path)

        # 定義目標四個角的坐標（左上、右上、左下、右下的順序）
        dst_points = np.array([[0, 0],
                               [1080, 0],
                               [0, 1080],
                               [1080, 1080]], dtype=np.float32)

        # 執行透視變換（homography）
        H, _ = cv2.findHomography(src_points, dst_points)

        # 對原始圖像應用透視變換
        warped_image = cv2.warpPerspective(image, H, (1080, 1080))

        # 儲存結果
        output_filename = os.path.join(output_directory, filename.replace('.jpg', '_front_view_slice.jpg'))
        cv2.imwrite(output_filename, warped_image)

print("Front view slice Homography transformation completed and saved successfully.")


Front view slice Homography transformation completed and saved successfully.
